In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
class TimeSeriesDataset2(Dataset):
    def __init__(self, csv_file, window_size, step_size):
        self.data_frame = pd.read_csv(csv_file) # 读取CSV文件
        self.window_size = window_size
        self.step_size = step_size
        self.samples = self._create_samples()

    def _create_samples(self):
        samples = []
        for start_pos in range(0, len(self.data_frame) - self.window_size + 1, self.step_size):
            end_pos = start_pos + self.window_size
            sample = self.data_frame.iloc[start_pos:end_pos].values
            samples.append(sample)
        return np.array(samples)

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        X = sample[:, :-1]  # 所有行，除了最后一列
        y = np.mean(sample[:, -1])  # 最后一列的平均值作为标签

        # 转换为Tensor
        X = torch.tensor(X, dtype=torch.float32)
        X = X.transpose(0, 1)
        y = torch.tensor(y, dtype=torch.float32)
        return X, y

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return x

class TransformerModel(nn.Module):
    def __init__(self, input_dim, num_heads, num_encoder_layers, dim_feedforward, max_seq_length, num_classes=1):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Linear(input_dim, dim_feedforward)
        self.pos_encoder = PositionalEncoding(dim_feedforward, max_seq_length)
        encoder_layer = nn.TransformerEncoderLayer(d_model=dim_feedforward, nhead=num_heads, dim_feedforward=dim_feedforward)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_encoder_layers)
        self.output_layer = nn.Linear(dim_feedforward, num_classes)

    def forward(self, src):
        src = src.transpose(0, 1)
        src = self.embedding(src) * math.sqrt(dim_feedforward)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src)
        output = self.output_layer(output.mean(dim=0))
        return output

In [ ]:
import time

use_cuda = torch.cuda.is_available()
device=torch.device('cuda' if use_cuda else 'cpu')
# 加载整个模型
model_path = 'transformer.pt'
model = torch.load(model_path)
model = model.to(device)

# 选择一个样本进行推理
SAMPLES_PER_GESTURE = 3000
step_size = 100
dataset = TimeSeriesDataset2('normalized_data-Copy1.csv', SAMPLES_PER_GESTURE,step_size)
sample, _ = dataset[110]  # 获取数据集的第一个样本
sample = sample.unsqueeze(0).to(device)  # 增加一个批处理维度

# 测量推理时间
start_time = time.time()
with torch.no_grad():  # 确保不计算梯度
    output = model(sample)
end_time = time.time()

# 打印推理时间和推理结果
print(f"Inference Time: {end_time - start_time:.6f} seconds")
predicted = output  # 获取概率最高的类别索引
print(f"Predicted Label: {predicted.item()}")